# Capybara Wikipedia Agent

This notebook demonstrates a Pydantic AI agent that can:
1. Fetch and analyze Wikipedia pages
2. Index multiple related pages
3. Search across the indexed content to answer questions

Based on week2/pydantic-ai-intro.ipynb and agents examples.


In [11]:
# Import required libraries
import requests
from pydantic_ai import Agent
from typing import List, Dict, Any
import json
from datetime import datetime


In [12]:
# Simple in-memory storage for fetched pages and summaries
page_store = {}
summaries_store = []


## Define Tools for the Agent


In [13]:
def fetch_web_page(url: str) -> str:
    """
    Fetch the content of a web page.
    
    Args:
        url: The URL of the web page to fetch
        
    Returns:
        The HTML content of the page, or an error message if fetch fails
    """
    try:
        # Add User-Agent header to avoid 403 errors from Wikipedia
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        # Store the page content
        page_store[url] = response.text
        
        # Return a summary of what was fetched
        return f"Successfully fetched {url}. Content length: {len(response.text)} characters. Page stored."
    except Exception as e:
        return f"Error fetching {url}: {str(e)}"


In [14]:
def save_summary(title: str, summary: str) -> str:
    """
    Save a summary to the summaries store.
    
    Args:
        title: Title of the document or page summarized
        summary: The summary text to save
        
    Returns:
        Confirmation message
    """
    entry = {
        "title": title,
        "summary": summary,
        "timestamp": datetime.now().isoformat()
    }
    summaries_store.append(entry)
    return f"Saved summary for: {title}"


In [15]:
def search_content(query: str) -> List[Dict[str, Any]]:
    """
    Search through all stored page content for relevant information.
    
    Args:
        query: The search query string
        
    Returns:
        List of search results with URL and relevant excerpts
    """
    results = []
    query_lower = query.lower()
    
    for url, content in page_store.items():
        content_lower = content.lower()
        if query_lower in content_lower:
            # Find the context around the match
            idx = content_lower.find(query_lower)
            start = max(0, idx - 200)
            end = min(len(content), idx + len(query) + 200)
            excerpt = content[start:end]
            
            results.append({
                "url": url,
                "excerpt": excerpt,
                "relevance": "high" if query_lower in content_lower[:500] else "medium"
            })
    
    return results


## Create the Agent

We'll use iterative prompt improvement to get the agent to use the tools correctly.


In [16]:
# Initial agent instructions - this will be refined based on behavior
instructions = """
You are a helpful research assistant that can fetch web pages, analyze content, and search through information.

IMPORTANT: When a user asks about a web page:
1. ALWAYS use the fetch_web_page tool first to get the page content
2. Read and summarize the content
3. Use the save_summary tool to store your summary
4. Report what you found to the user

When searching for information:
1. Use the search_content tool with relevant keywords from the question
2. Look through the results carefully
3. Synthesize information from multiple sources
4. Provide a comprehensive answer citing your sources

Be thorough and use the tools available to you. Don't skip tool usage.
"""


In [17]:
# Create the agent with all tools
agent = Agent(
    name='research_assistant',
    instructions=instructions,
    tools=[fetch_web_page, save_summary, search_content],
    model='openai:gpt-4o-mini'
)


## Helper: Log Tool Calls

First, let's add a helper to see what the agent is doing.


In [18]:
# Helper function to log tool calls - must be async
async def log_function_calls(run_ctx, event_stream):
    """Log all function calls and their results"""
    async for event in event_stream:
        try:
            if hasattr(event, 'is_function_call') and event.is_function_call():
                print(f"🔧 TOOL CALL: {event.content.function_name}({event.content.arguments})")
            elif hasattr(event, 'is_function_result') and event.is_function_result():
                result = event.content.result
                if isinstance(result, str) and len(result) > 200:
                    print(f"✅ TOOL RESULT: {result[:200]}...")
                else:
                    print(f"✅ TOOL RESULT: {result}")
            elif hasattr(event, 'is_response') and event.is_response():
                print(f"📝 RESPONSE: {event.content}")
        except Exception as e:
            print(f"⚠️ Error logging event: {e}")


In [19]:
## Question 5: Fetch and Analyze a Wikipedia Page


In [ ]:
# Clear storage for clean test
page_store.clear()
summaries_store.clear()

# Ask the question
question = "Provide a short summary of this page and store that summary: https://en.wikipedia.org/wiki/Capybara"

print(f"Question: {question}")
print("\nAgent processing...\n")

# Run with event logging
result = await agent.run(user_prompt=question, event_stream_handler=log_function_calls)

print("\n" + "="*80)
print("Final Response:")
print("="*80)
print(result.output)
print("="*80)

# Verify what tools were actually used
print("\n" + "="*80)
print("Verification:")
print(f"Pages fetched: {list(page_store.keys())}")
print(f"Summaries saved: {len(summaries_store)}")
if summaries_store:
    for summary in summaries_store:
        print(f"  - {summary['title']}")
print("="*80)


Question: Provide a short summary of this page and store that summary: https://en.wikipedia.org/wiki/Capybara

Agent processing...


Final Response:
I have summarized the Wikipedia page on Capybaras. Here's the summary:

**Capybara**: The Capybara (Hydrochoerus hydrochaeris) is the largest rodent in the world, native to South America. These semi-aquatic mammals are found in groups near bodies of water and can weigh up to 66 kilograms (146 lb). Capybaras have a social structure, living in groups of 10 to 20 individuals, and are herbivorous, feeding mainly on grasses and aquatic plants. They have adaptations for swimming, including webbed feet and a streamlined body. Their conservation status is currently listed as Least Concern, although habitat loss and hunting pose threats.

This summary has been stored successfully.


In [ ]:
# Display all saved summaries
print("="*80)
print("ALL SAVED SUMMARIES:")
print("="*80)
for i, summary in enumerate(summaries_store, 1):
    print(f"\n{i}. {summary['title']}")
    print(f"   {summary['summary'][:200]}...")
    print(f"   Timestamp: {summary['timestamp']}")
print("="*80)


In [29]:
# Call for Question 6 
# Clear storage for clean test
page_store.clear()
summaries_store.clear()

# Ask the question
question = '''Provide a short summary of the following pages and stores those summaryies:

 Lesser capybara — https://en.wikipedia.org/wiki/Lesser_capybara

Hydrochoerus (genus) — https://en.wikipedia.org/wiki/Hydrochoerus

Neochoerus (extinct genus related to capybaras) — https://en.wikipedia.org/wiki/Neochoerus

Caviodon (extinct genus of rodents related to capybaras) — https://en.wikipedia.org/wiki/Caviodon

Neochoerus aesopi (extinct species close to capybaras) — https://en.wikipedia.org/wiki/Neochoerus_aesopi

'''

print(f"Question: {question}")
print("\nAgent processing...\n")

# Run with event logging
result = await agent.run(user_prompt=question, event_stream_handler=log_function_calls)

print("\n" + "="*80)
print("Final Response:")
print("="*80)
print(result.output)
print("="*80)

# Verify what tools were actually used
print("\n" + "="*80)
print("Verification:")
print(f"Pages fetched: {list(page_store.keys())}")
print(f"Summaries saved: {len(summaries_store)}")
if summaries_store:
    for summary in summaries_store:
        print(f"  - {summary['title']}")
print("="*80)


Question: Provide a short summary of the following pages and stores those summaryies:

 Lesser capybara — https://en.wikipedia.org/wiki/Lesser_capybara

Hydrochoerus (genus) — https://en.wikipedia.org/wiki/Hydrochoerus

Neochoerus (extinct genus related to capybaras) — https://en.wikipedia.org/wiki/Neochoerus

Caviodon (extinct genus of rodents related to capybaras) — https://en.wikipedia.org/wiki/Caviodon

Neochoerus aesopi (extinct species close to capybaras) — https://en.wikipedia.org/wiki/Neochoerus_aesopi



Agent processing...


Final Response:
I have successfully fetched and summarized the requested pages. Here are the summaries:

1. **Lesser Capybara**: The lesser capybara (Hydrochoerus isthmius) is a smaller species of capybara found in Colombia and Venezuela. Typically around 75 cm long, they inhabit wetlands, are social creatures living in groups, and primarily feed on grasses and aquatic plants.

2. **Hydrochoerus (genus)**: Hydrochoerus is a genus of large rodents in the C

In [ ]:
# Call for Question 6b 
# DON'T clear storage - we want to keep the indexed pages from Question 6a
# This will search through all previously indexed pages

# Ask the question about threats
question = "What are the threats to capybara populations?"

print(f"Question: {question}")
print("\nAgent processing...\n")

# Run with event logging
result = await agent.run(user_prompt=question, event_stream_handler=log_function_calls)

print("\n" + "="*80)
print("Final Response:")
print("="*80)
print(result.output)
print("="*80)

# Verify what tools were actually used
print("\n" + "="*80)
print("Verification:")
print(f"Pages fetched: {list(page_store.keys())}")
print(f"Summaries saved: {len(summaries_store)}")
if summaries_store:
    for summary in summaries_store:
        print(f"  - {summary['title']}")
print("="*80)


Question: 

Based on the summaries you created and the following pages,
provide a short summary of the threats to the capybara populations.

Capybara — https://en.wikipedia.org/wiki/Capybara

Lesser capybara — https://en.wikipedia.org/wiki/Lesser_capybara

Hydrochoerus (genus) — https://en.wikipedia.org/wiki/Hydrochoerus

Neochoerus (extinct genus related to capybaras) — https://en.wikipedia.org/wiki/Neochoerus

Caviodon (extinct genus of rodents related to capybaras) — https://en.wikipedia.org/wiki/Caviodon

Neochoerus aesopi (extinct species close to capybaras) — https://en.wikipedia.org/wiki/Neochoerus_aesopi



Agent processing...


Final Response:
Based on the information gathered from the Wikipedia pages, here is a summary of the threats to capybara populations:

1. **Habitat Loss**: Capybaras face significant threats from habitat destruction, primarily due to agricultural expansion and urban development. As their natural environments are altered or destroyed, their populations d